In [1]:
import igraph as ig
from igraph import *
import numpy as np
import math
import random
import time
import copy
###################################################
def init_IMDB_graph():
    N=21           #Number_of_vertices
    g=Graph()
    g.add_vertices(N)
    g.add_edges([(0,1), (0,2), (0,4), (0,5), (0,6), (0,11),(0,13), (0,17), (0,20), (1,2), (2,4), (2,5), (2,6), (2,11), (2,13), (2,17), (2,20),
                (3,4), (3,5), (3,9), (4,5), (4,6), (4,11), (4,13), (4,17), (4,20), (5,6), (5,11), (5,13), (5,17), (5,20), (6,7), (6,8), (6,9),
                (6,10), (6,16), (6,17), (6,20), (8,9), (8,10), (11,12), (11,13), (11,17), (11,20), (13,14), (13,15), (13,17),(13,20), (17,18),
                (17,20), (19,20)])
    g.vs["table_name"] = ["title","kind_type","aka_title","info_type","movie_info","movie_info_idx","cast_info","role_type","name",
                           "person_info","aka_name","movie_keyword","keyword","movie_companies","company_type","company_name","char_name",
                           "complete_cast","comp_cast_type","link_type","movie_link"]
    g.vs["size"]=[4189165, 7, 515643, 113, 257641243, 2280011, 61375706, 12, 6278675, 4048009, 1279133, 7200278, 222746, 4784207, 4,
                   352569, 2763179, 135086, 4, 18, 2530092]
    g.vs["label"]=["0","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20"]
    g.es["selectivity"]=[0.142, 2.01e-7, 2.14e-7, 2.16e-7, 2.22e-7, 2.11e-7, 2.17e-7, 2.20e-7, 6.88e-9, 0.1428, 1.37e-6, 9.26e-7, 2.01e-7,
                        2.73e-6, 1.25e-6, 2.13e-6, 2.74e-6, 8.84e-3, 8.84e-3, 8.84e-3, 6.00e-8, 9.01e-8, 2.38e-7, 3.08e-8, 1.58e-7, 5.08e-6,
                        4.01e-7, 9.84e-7, 6.25e-7, 9.75e-7, 8.91e-7, 3.26e-3, 1.59e-7, 1.38e-6, 1.60e-7, 1.62e-8, 1.58e-8, 7.58e-7, 1.59e-7,
                        1.59e-7, 4.48e-6, 1.75e-6, 2.57e-6, 3.28e-6, 0.25, 2.83e-6, 1.12e-5, 1.28e-6, 2.57e-6, 2.57e-6, 5.55e-2]
    g.es["label"]=[0.142, 2.01e-7, 2.14e-7, 2.16e-7, 2.22e-7, 2.11e-7, 2.17e-7, 2.20e-7, 6.88e-9, 0.1428, 1.37e-6, 9.26e-7, 2.01e-7,
                        2.73e-6, 1.25e-6, 2.13e-6, 2.74e-6, 8.84e-3, 8.84e-3, 8.84e-3, 6.00e-8, 9.01e-8, 2.38e-7, 3.08e-8, 1.58e-7, 5.08e-6,
                        4.01e-7, 9.84e-7, 6.25e-7, 9.75e-7, 8.91e-7, 3.26e-3, 1.59e-7, 1.38e-6, 1.60e-7, 1.62e-8, 1.58e-8, 7.58e-7, 1.59e-7,
                        1.59e-7, 4.48e-6, 1.75e-6, 2.57e-6, 3.28e-6, 0.25, 2.83e-6, 1.12e-5, 1.28e-6, 2.57e-6, 2.57e-6, 5.55e-2]
        
    return g,N
###################################################
def init_test_graph():
    N=7            #Number_of_vertices
    g=Graph()
    g.add_vertices(N)
    g.add_edges([(0,1), (0,6), (1,2), (1,4), (2,3), (2,4), (3,4), (5,6)]) # id = label-1
    g.vs["size"]=[10, 10, 10, 10, 10, 10, 10]
    g.vs["label"]=["1","2","3","4","5","6","7"]
    g.vs["table_name"]=["1","2","3","4","5","6","7"]
    g.es["selectivity"]=[0.7, 0.9, 0.3, 0.4, 0.5, 0.6, 0.2, 0.8]
    g.es["label"]=[0.7, 0.9, 0.3, 0.4, 0.5, 0.6, 0.2, 0.8]

    return g,N                    
###################################################
def produce_sel_pred(g,N):
    sel=np.zeros((N,N))
    pred=np.zeros((N,N), np.bool)
    for edge in g.es:
        s=edge.source
        t=edge.target
        sel[t][s]=sel[s][t]=edge["selectivity"]
        pred[t][s]=pred[s][t]=True
        
    return sel,pred
###################################################
def merge_nodes ( g , sel , pred , i , j ):
    new_size= g.vs[i]["size"] * g.vs[j]["size"] * g.es[g.get_eid(i,j)]["selectivity"]
    sel[i][j] = sel[j][i] = 0
    pred[i][j] = pred[j][i] = False
    g.delete_edges(g.get_eid(i,j))
        
    for k in g.neighbors(i):
        if pred[k][j] :
                
            sel[i][k] = sel[k][i] = sel[i][k] * sel[j][k]
            g.es[g.get_eid(i,k)]["selectivity"]= sel[i][k]                                  # set selectivity
            g.es[g.get_eid(i,k)]["label"]= sel[i][k]                                        # set label
                
            sel[k][j] = sel[j][k] = 0
            pred[k][j] = pred[j][k] = False
            g.delete_edges(g.get_eid(k,j))
                
    for m in g.neighbors(j):
            
        pred[i][m] = pred[m][i] = True
        sel[i][m] = sel[m][i] = sel[j][m]
        g.add_edge(i,m)
        g.es[g.get_eid(i,m)]["selectivity"]= sel[j][m]                                      # set selectivity
        g.es[g.get_eid(i,m)]["label"]= sel[j][m]                                            # set label
            
        sel[m][j] = sel[j][m] = 0
        pred[m][j] = pred[j][m] = False
        g.delete_edges(g.get_eid(m,j))
                        
    g.vs[i]["label"] =  g.vs[i]["label"] + "," + g.vs[j]["label"]                           # set label
    g.vs[i]["table_name"] =  g.vs[i]["table_name"] + "," + g.vs[j]["table_name"]            # set name
    g.vs[i]["size"]= math.ceil(new_size)                                                    # set size
        
    sel = np.delete(sel, j , axis=1)
    sel = np.delete(sel, j, axis=0)
    pred = np.delete(pred, j , axis=1)
    pred = np.delete(pred, j, axis=0)
    g.delete_vertices(j)
    
    return g , sel , pred
###################################################
def GOO_Algorithm (N,g,sel,pred):
    label_steps=[]
    name_steps=[]
    for n in range (N-1,0,-1):
        min_value= -1
        for edge in g.es:          #find min value
            s=edge.source
            t=edge.target
            if s == t:
                continue
            val= g.vs[s]["size"] * g.vs[t]["size"] * edge["selectivity"]
            if min_value == -1:
                min_value = val
                i=s
                j=t
            elif val < min_value:
                min_value = val
                i=s
                j=t
        
        label_steps.append((g.vs[i]["label"] ,g.vs[j]["label"] , min_value ))
        name_steps.append((g.vs[i]["table_name"] ,g.vs[j]["table_name"] , min_value ))
        g,sel,pred = merge_nodes(g,sel,pred,i,j)
        
#         pic_name="step"+str(N-n)+".png"
#         plot_graph(g.copy(),pic_name)
    
    return label_steps , name_steps  
###################################################
def REC_BF_Algorithm(n,g,sel,pred,label_steps,name_steps,cost):
    if n==1:       
        global final_min_cost
        global final_label_steps
        global final_name_steps
        if final_min_cost == -1:
            final_min_cost = cost
            final_label_steps = label_steps
            final_name_steps = name_steps
        elif final_min_cost > cost:
            final_min_cost = cost
            final_label_steps = label_steps 
            final_name_steps = name_steps 
        
    else:
        for edge in g.es:
            g_copy=g.copy()
            sel_copy=copy.deepcopy(sel)
            pred_copy=copy.deepcopy(pred)
            cost_copy=cost
            label_steps_copy=label_steps.copy()
            name_steps_copy=name_steps.copy()
            n_copy=n
            s=edge.source
            t=edge.target
            step_cost= g_copy.vs[s]["size"] * g_copy.vs[t]["size"] * edge["selectivity"]
            label_steps_copy.append((g_copy.vs[s]["label"] ,g_copy.vs[t]["label"] , step_cost ))
            name_steps_copy.append((g_copy.vs[s]["table_name"] ,g_copy.vs[t]["table_name"] , step_cost ))
            cost_copy += step_cost
            g_copy,sel_copy,pred_copy = merge_nodes(g_copy,sel_copy,pred_copy,s,t)
            REC_BF_Algorithm(n-1,g_copy,sel_copy,pred_copy,label_steps_copy,name_steps_copy,cost_copy)        
                   
###################################################
def random_subgraph(g , N , min_size , max_size):
    candid=[]
    closed=[]
    start_v = random.randint(0,N-1)
    candid.append(start_v)
    cand_size=1
        
    if N<max_size:
        subg_N= random.randint (min_size,N)
    else:
        subg_N= random.randint(min_size,max_size)
            
    for count in range(subg_N):
         sel_index= random.randint(0,cand_size-1)
         for neib in g.neighbors(candid[sel_index]):
             if neib not in closed and neib not in candid:
                 candid.append(neib)
                 cand_size+=1
                
         closed.append(candid.pop(sel_index))
         cand_size-=1
        
    return g.induced_subgraph(closed) , subg_N
###################################################
def plot_graph(g,pic_name):
    for node in g.vs:
        node["label"] = node["label"] + "(" + str(node["size"]) + ")" 
    visual_style = {}
    out_name = pic_name
    # Set bbox and margin
    visual_style["bbox"] = (1900,1000)
#     visual_style["bbox"] = (1100,600)           # No need to move
    visual_style["margin"] = 27
    # Set vertex colours
    visual_style["vertex_color"] = 'yellow'
    # Set vertex size
    visual_style["vertex_size"] = 90
    # Set vertex lable size
    visual_style["vertex_label_size"] = 15
    # Don't curve the edges
    visual_style["edge_curved"] = False
    # Set the layout
    my_layout = g.layout_lgl()
    visual_style["layout"] = my_layout
    # Plot the graph
    ig.plot(g, out_name, **visual_style)
###################################################
def print_results(subg , subg_N, label_steps , name_steps , time , alg , mode):
    if alg== "GOO":
        alg="GREEDY OPERATOR ORDERING"
    elif alg== "BF":
        alg="RECURSIVE BRUTE FORCE"
    print("-----------------------------------------------------------------------------------------------------------------")
    print("                                " ,alg , "ALGORITHM:                                                                    ")
    print("                                     (", mode , "BASED RESULTS )                                                                   ")
    print("-----------------------------------------------------------------------------------------------------------------")
    print(" ")
    print("SUB GRAPH NUMBER OF EDGES:" , len(subg.es()))
    print("SUB GRAPH NUMBER OF NODES:" , subg_N)
    print("SUB GRAPH NODES:", end=" ")
    total_cost=0
    for x in range (len(subg.vs) -1 ):
        if mode == "LABEL":
            print (subg.vs[x]["label"] ,"," , end=" ")
        elif mode == "NAME":
            print (subg.vs[x]["table_name"] ,",", end=" ")
    if mode == "LABEL":
        print (subg.vs[-1]["label"])
    elif mode == "NAME":
        print (subg.vs[-1]["table_name"])        
    print("OPTIMAL STEPS FOR JOIN:")
    step_count=1
    if mode == "LABEL":
        for s in label_steps:
            print("Step" ,step_count,":", "(" , s[0],")" ,"⨝", "(" , s[1],")" , "    " , "COST:" , s[2])
            step_count+=1
            total_cost += s[2]
    if mode == "NAME":
        for s in name_steps:
            print("Step" ,step_count,":", "(" , s[0],")" ,"⨝", "(" , s[1],")" , "    " , "COST:" , s[2])
            step_count+=1
            total_cost += s[2]
    print(" ")
    print("EXECUTION TIME:" , time , "ns")
    print("TOTAL COST:" , total_cost)
    print(" ")
   
###################################################
g,N=init_IMDB_graph()
# g,N=init_test_graph()
subg , subg_N = random_subgraph(g , N , 6 , 11)
# subg , subg_N =g,N
plot_graph(subg.copy(),"graph.png")
sel,pred=produce_sel_pred(subg,subg_N)
#GOO
goo_t1= time.time_ns()
l_steps , n_steps = GOO_Algorithm(subg_N,subg.copy(),copy.deepcopy(sel),copy.deepcopy(pred))
GOO_time=time.time_ns()-goo_t1
print_results(subg , subg_N, l_steps , n_steps , GOO_time , "GOO" , "LABEL")
#REC_BF
bf_t1= time.time_ns()
final_label_steps=[]
final_name_steps=[]
final_min_cost=-1
REC_BF_Algorithm(subg_N,subg.copy(),copy.deepcopy(sel),copy.deepcopy(pred),[],[],0)
BF_time= time.time_ns() - bf_t1
print_results(subg , subg_N, final_label_steps , final_name_steps , BF_time , "BF" , "LABEL")

-----------------------------------------------------------------------------------------------------------------
                                 GREEDY OPERATOR ORDERING ALGORITHM:                                                                    
                                     ( LABEL BASED RESULTS )                                                                   
-----------------------------------------------------------------------------------------------------------------
 
SUB GRAPH NUMBER OF EDGES: 9
SUB GRAPH NUMBER OF NODES: 6
SUB GRAPH NODES: 3 , 4 , 6 , 9 , 17 , 20
OPTIMAL STEPS FOR JOIN:
Step 1 : ( 6 ) ⨝ ( 17 )      COST: 130997.77820731279
Step 2 : ( 4 ) ⨝ ( 6,17 )      COST: 0.4804651906716073
Step 3 : ( 4,6,17 ) ⨝ ( 20 )      COST: 2.50381567893216e-11
Step 4 : ( 3 ) ⨝ ( 4,6,17,20 )      COST: 0.99892
Step 5 : ( 3,4,6,17,20 ) ⨝ ( 9 )      COST: 0.0493824713928
 
EXECUTION TIME: 0 ns
TOTAL COST: 130999.30697497488
 
---------------------------------------------